In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# Load data
df = pd.read_csv('BigBasket Products.csv', index_col='index')

# Initialize Sentence Transformer model for advanced text embeddings
text_model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate text embeddings for descriptions
description_embeddings = text_model.encode(df['description'].tolist())

# Preprocessing pipeline for categorical/numerical features
preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), 
     ['category', 'sub_category', 'brand', 'type']),
    ('num', MinMaxScaler(), ['sale_price', 'market_price', 'rating'])
])

# Transform features
processed_features = preprocessor.fit_transform(df)

# Combine all features
combined_features = np.hstack([
    processed_features,
    description_embeddings
])

# Compute similarity matrix
cosine_sim = cosine_similarity(combined_features)

# Recommendation function
def get_recommendations(product_name, n=5):
    try:
        idx = df[df['product'] == product_name].index[0]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_indices = [i[0] for i in sim_scores[1:n+1]]
        return df[['product', 'category', 'sub_category']].iloc[sim_indices]
    except:
        return "Product not found in database"

C:\Users\Aryan\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Aryan\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Aryan\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an adminis

TypeError: 'float' object is not subscriptable

In [ ]:
print(get_recommendations("Garlic Oil - Vegetarian Capsule 500 mg"))